In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import keras.backend as K
import tensorflow as tf

Using TensorFlow backend.


In [3]:
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
# config.gpu_options.visible_device_list = "0" #only the gpu 0 is allowed
set_session(tf.Session(config=config))

In [4]:
from __future__ import print_function
from __future__ import absolute_import

import warnings
import numpy as np

from keras.preprocessing import image
from keras.applications import VGG16
# from keras.applications import Xception, VGG16
import keras

In [5]:
from keras.applications import InceptionV3


In [6]:
model = keras.models.Sequential([
    VGG16(include_top=False, weights="imagenet", input_shape=(128, 128, 3)),
    keras.layers.Flatten(),
#     keras.layers.Dense(128,activation = 'relu',kernel_initializer='random_normal'),
#     keras.layers.Dense(16,activation = 'relu',kernel_initializer='random_normal'),
    keras.layers.Dense(1,activation = 'sigmoid')
])

# model = keras.applications.VGG16(include_top=True, weights='imagenet', input_shape=(128,128,3), classes=2)
    

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 8193      
Total params: 14,722,881
Trainable params: 14,722,881
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.load_weights("VGG16_keras.h5")


In [8]:
model.compile(optimizer = "adam", loss = 'binary_crossentropy',metrics = ['accuracy'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
from keras.preprocessing.image import ImageDataGenerator
batch_size = 32

In [10]:
dataGenerator = ImageDataGenerator(rescale=1./255,rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,shear_range=0.05)

In [11]:
generator = dataGenerator.flow_from_directory(
        '/hdd/tam/kaggle/train_imgs/',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',shuffle=True)

Found 637965 images belonging to 2 classes.


In [12]:
test_generator = dataGenerator.flow_from_directory(
        '/hdd/tam/kaggle/test_imgs/',
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='binary',
        subset='training',shuffle=True)

Found 3879 images belonging to 2 classes.


In [13]:
from collections import Counter
from sklearn.utils import class_weight

counter = Counter(generator.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     

class_weights_2 = class_weight.compute_class_weight(
               'balanced',
                np.unique(generator.classes), 
                generator.classes)


In [14]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir="./log_vgg16_keras")
checkpoints = keras.callbacks.ModelCheckpoint("./log_vgg16_keras/checkpoint_{epoch:04d}.pth", monitor='val_loss', verbose=0, save_best_only=True, period=10)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0)

In [ ]:
model.fit_generator(generator,validation_data=test_generator,steps_per_epoch=int(637965/batch_size), epochs=100,workers=1,validation_steps=3879/batch_size,class_weight = class_weights,callbacks = [tensorboard_callback,checkpoints])




Epoch 1/100
15200/19936 [=====================>........] - ETA: 19:38 - loss: 12.4917 - acc: 0.2164

In [38]:
model.evaluate_generator(test_generator,3879/batch_size, workers=8)


[1.3823501131343423, 0.6643464812976732]

In [39]:
model.save_weights("VGG16_keras.h5")
